In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [3]:
huggingface_dataset_name = "squad"
dataset = load_dataset(huggingface_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [4]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print(f"Example {i + 1}")

    print(dash_line)
    print(f"context: \n{dataset['train'][index]['context']}")
    print(f"question: \n{dataset['train'][index]['question']}")
    print(f"answers: \n{dataset['train'][index]['answers']['text'][0]}")


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
context: 
Notre Dame is known for its competitive admissions, with the incoming class enrolling in fall 2015 admitting 3,577 from a pool of 18,156 (19.7%). The academic profile of the enrolled class continues to rate among the top 10 to 15 in the nation for national research universities. The university practices a non-restrictive early action policy that allows admitted students to consider admission to Notre Dame as well as any other colleges to which they were accepted. 1,400 of the 3,577 (39.1%) were admitted under the early action plan. Admitted students came from 1,311 high schools and the average student traveled more than 750 miles to Notre Dame, making it arguably the most representative university in the United States. While all entering students begin in the College of

In [6]:
model_name = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

### 零样本微调

In [7]:
for i, index in enumerate(example_indices):
    context = dataset['train'][index]['context']
    question = dataset['train'][index]['question']
    answer = dataset['train'][index]['answers']['text'][0]

    prompt = f"""
Given the following context and an certain answer,
please generate a certain question corresponding to the context and the answer:
Context: {context}
Answer: {answer}
Question:
"""

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = tokenizer.decode(original_model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True)

    print(dash_line)
    print(f"Example {i+1}")
    print(f"context: \n{context}")
    print(f"question: \n{question}")
    print(f"answer: \n{answer}")
    print(f"Generated question: \n{outputs}")

---------------------------------------------------------------------------------------------------
Example 1
context: 
Notre Dame is known for its competitive admissions, with the incoming class enrolling in fall 2015 admitting 3,577 from a pool of 18,156 (19.7%). The academic profile of the enrolled class continues to rate among the top 10 to 15 in the nation for national research universities. The university practices a non-restrictive early action policy that allows admitted students to consider admission to Notre Dame as well as any other colleges to which they were accepted. 1,400 of the 3,577 (39.1%) were admitted under the early action plan. Admitted students came from 1,311 high schools and the average student traveled more than 750 miles to Notre Dame, making it arguably the most representative university in the United States. While all entering students begin in the College of the First Year of Studies, 25% have indicated they plan to study in the liberal arts or social scie

### 少样本微调

In [11]:
def make_prompt(example_indices_full, example_index_to_questions):
    prompt = """
Given the following context and an certain answer,
please generate a question corresponding to the context and the answer:
"""

    for index in example_indices_full:
        context = dataset['train'][index]['context']
        question = dataset['train'][index]['question']
        answer = dataset['train'][index]['answers']['text'][0]
        prompt += f"""
Context: {context}
Answer: {answer}
Question: {question}
"""

    context = dataset['train'][example_index_to_questions]['context']
    question = dataset['train'][example_index_to_questions]['question']
    answer = dataset['train'][example_index_to_questions]['answers']['text'][0]
    prompt += f"""
Context: {context}
Answer: {answer}
Question:
"""

    return prompt

example_indices_full = [50, 100]
example_index_to_questions = 150
few_shot_prompt = make_prompt(example_indices_full, example_index_to_questions)
print(few_shot_prompt)


Given the following context and an certain answer,
please generate a question corresponding to the context and the answer:

Context: Father Joseph Carrier, C.S.C. was Director of the Science Museum and the Library and Professor of Chemistry and Physics until 1874. Carrier taught that scientific research and its promise for progress were not antagonistic to the ideals of intellectual and moral culture endorsed by the Church. One of Carrier's students was Father John Augustine Zahm (1851–1921) who was made Professor and Co-Director of the Science Department at age 23 and by 1900 was a nationally prominent scientist and naturalist. Zahm was active in the Catholic Summer School movement, which introduced Catholic laity to contemporary intellectual issues. His book Evolution and Dogma (1896) defended certain aspects of evolutionary theory as true, and argued, moreover, that even the great Church teachers Thomas Aquinas and Augustine taught something like it. The intervention of Irish Ameri

In [12]:
context = dataset['train'][example_index_to_questions]['context']
question = dataset['train'][example_index_to_questions]['question']
answer = dataset['train'][example_index_to_questions]['answers']['text'][0]

inputs = tokenizer(few_shot_prompt, return_tensors="pt")
outputs = tokenizer.decode(original_model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True)

print(dash_line)
print(f"context: \n{context}")
print(f"question: \n{question}")
print(f"answer: \n{answer}")
print(f"Generated question: \n{outputs}")

---------------------------------------------------------------------------------------------------
context: 
Since the construction of its oldest buildings, the university's physical plant has grown substantially. Over the years 29 residence halls have been built to accommodate students and each has been constructed with its own chapel. Many academic building were added together with a system of libraries, the most prominent of which is the Theodore Hesburgh Library, built in 1963 and today containing almost 4 million books. Since 2004, several buildings have been added, including the DeBartolo Performing Arts Center, the Guglielmino Complex, and the Jordan Hall of Science. Additionally, a new residence for men, Duncan Hall, was begun on March 8, 2007, and began accepting residents for the Fall 2008 semester. Ryan Hall was completed and began housing undergraduate women in the fall of 2009. A new engineering building, Stinson-Remick Hall, a new combination Center for Social Concerns/I